<a href="https://colab.research.google.com/github/adnan119/Fashion-mnist-GANs/blob/main/conditional_fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose #<-- transposed convolution for upsampling
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import Reduction
from tensorflow import stop_gradient
from tensorflow import one_hot
from tensorflow import concat
from tensorflow import tile
from tensorflow import stop_gradient
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import argparse
import cv2
import os

In [1]:
class DCGAN:
	@staticmethod
	def build_generator(dim, depth, channels=1, inputDim=100,
		outputDim=512):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (dim, dim, depth)
		chanDim = -1

    # first set of FC => RELU => BN layers
		model.add(Dense(input_dim=inputDim, units=outputDim))
		model.add(Activation("relu"))
		model.add(BatchNormalization()) # <-- batch normalization layers will be used to reduce internal covariate shift
  
		# second set of FC => RELU => BN layers, this time preparing
		# the number of FC nodes to be reshaped into a volume
		model.add(Dense(dim * dim * depth))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
  
    # reshape the output of the previous layer set, upsample +
		# apply a transposed convolution, RELU, and BN
		model.add(Reshape(inputShape))
		model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2),
			padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
  
    # apply another upsample and transposed convolution, but
		# this time output the TANH activation
		model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2),
			padding="same"))
		model.add(Activation("tanh"))
		# return the generator model
		return model

	@staticmethod
	def build_discriminator(width, height, depth, alpha=0.2):
		# initialize the model along with the input shape to be
		# "channels last"
		model = Sequential()
		inputShape = (height, width, depth)
		# first set of CONV => RELU layers
		model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2),
			input_shape=inputShape))
		model.add(LeakyReLU(alpha=alpha))
		# second set of CONV => RELU layers
		model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
		model.add(LeakyReLU(alpha=alpha))
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(LeakyReLU(alpha=alpha))
		# sigmoid layer outputting a single value
		model.add(Dense(1))
		model.add(Activation("sigmoid"))
		# return the discriminator model
		return model

In [2]:
from numpy import zeros
from numpy.random import randn
from matplotlib import pyplot

def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 

def generate_fake_samples(g_model, latent_dim, n_samples):
  # generate points in latent space
  x_input = generate_latent_points(latent_dim, n_samples)
  # predict outputs
  X = g_model.predict(x_input)
  images = ((X * 127.5) + 127.5).astype("uint8")
  images = np.repeat(images, 3, axis=-1)
  # create 'fake' class labels (0)
  y = zeros((n_samples, 1))
  return X, y
 

# **Part - 2 Conditional GANs**
# Adding Conditional Generation functionality to our GAN architecture

The GAN architecture that we created above is an example of unconditional Generations i.e. the entire architecture is generated without passing any information related to the class labels.

As you might have noticed above we haven't used any labels while training either the discriminator or the generator.

With unconditional Generation we get output from random classes, on the other hand **conditional generation** allows us to ask for an example from a specific class and you get what you ask for, i.e. a random example from a class you specify.




In conditional GANs, the input vector for the generator will need to include the class information. The class is represented using a one-hot encoded vector where its length is the number of classes and each index represents a class. The vector is all 0's and a 1 on the chosen class.
We'll combine/concatenate this one-hot vector with the random noise to create a new vector to be passed into the generator network.

In [ ]:
def get_one_hot_labels(labels, n_classes):
    return one_hot(labels, n_classes)

Next, we need to be able to concatenate the one-hot class vector to the noise vector before giving it to the generator. We will also need to do this when adding the class channels to the discriminator.


In [ ]:
def combine_vectors(x, y, dim=1):

    combined = concat((x,y),dim)
    
    return combined

In [ ]:
fmnist_shape = (1, 28, 28)
n_classes = 10
input_dim = 7

#initializing a few hyper-parameters
NUM_EPOCHS = 50
BATCH_SIZE = 128
INIT_LR = 2e-4

In [ ]:
# load the Fashion MNIST dataset and stack the training and testing
# data points so we have additional training data

((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
trainImages = np.concatenate([trainX, testX])

labels = np.concatenate([trainY, testY])
# add in an extra dimension for the channel and scale the images
# into the range [-1, 1] (which is the range of the tanh
# function)
trainImages = np.expand_dims(trainImages, axis=1)
trainImages = (trainImages.astype("float") - 127.5) / 127.5

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
trainImages.shape[-1]

28

Now we'll initialize our new generator and discriminator architecture, the main changes that we'd need make is mostly regarding the inputs to both the architectures.

###Generator Input

To pass the class information to the generator we'll pass the noise vector & concatenate the one-hot labels, the one-hot labels here will have one for the corresponding label and zeros elsewhere.

So now the dimension of the new input to the generator will be: **length of the noise vector + number of classes** 

###Discriminator Input 
Now to pass the class information to the Dicriminator we'll create one-hot matrices which we'll concatenate along with the channel(s) of the input images. This input will then be used to train the discriminator/classifier to classify images based on not just real and fake but also based on their class labels provided by the one-hot matrices.

Input dimnesion to the discriminator will be: **number of image channels + number of classes**



In [ ]:
def get_input_dimensions(input_dim, fmnist_shape, n_classes):

    generator_input_dim = input_dim + n_classes
    discriminator_im_chan = n_classes+fmnist_shape[0]

    return generator_input_dim, discriminator_im_chan


generator_input_dim, discriminator_im_chan = get_input_dimensions(input_dim, fmnist_shape, n_classes)

model = DCGAN()
conditional_generator = model.build_generator(7,64, channels=1, inputDim=generator_input_dim)
conditional_discriminator = model.build_discriminator(28,28,discriminator_im_chan)

In [ ]:
#initializing the discriminator optimizer & loss function
discOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
conditional_discriminator.compile(loss=BinaryCrossentropy(from_logits=True,
    reduction=Reduction.NONE), optimizer=discOpt)

# build the adversarial model by first setting the discriminator to
# *not* be trainable, then combine the generator and discriminator
# together
print("[INFO] building GAN...")
# conditional_discriminator.trainable = False
# ganInput = Input(shape=(generator_input_dim,))
# ganOutput = conditional_discriminator(conditional_generator(ganInput))
# gan = Model(ganInput, ganOutput)
# compile the GAN
ganOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
conditional_generator.compile(loss=BinaryCrossentropy(from_logits=True, reduction=Reduction.NONE), optimizer=discOpt)

[INFO] building GAN...


In [ ]:
import tensorflow as tf
def update_gan(gan_output,labels):
  with tf.GradientTape() as tape:

    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    disc_input = combine_vectors(gan_output,image_one_hot_labels, dim=-1)
    disc_output = conditional_discriminator.predict(disc_input)
    gen_loss = bce(disc_output,tf.ones_like(disc_output))

    var_list = conditional_generator.trainable_weights
    gan_gradients = tape.gradient(gen_loss, var_list)

    discOpt.apply_gradients(zip(gan_gradients,var_list))
  # discOpt.minimize(gen_loss, var_list = conditional_generator.trainable_weights)
  # discOpt.apply_gradients(zip(gan_gradients, conditional_generator.trainable_variables))

In [ ]:
# loop over the epochs
for epoch in range(0, NUM_EPOCHS):
  # show epoch information and compute the number of batches per
  # epoch
  print("[INFO] starting epoch {} of {}...".format(epoch + 1,
    NUM_EPOCHS))
  batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)
  # loop over the batches
  for i in range(0, batchesPerEpoch):

    labelsBatch = labels[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
    imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
    # initialize an (empty) output path
    one_hot_labels = get_one_hot_labels(labelsBatch, n_classes)
    image_one_hot_labels = one_hot_labels[:, None, None, :]
    image_one_hot_labels = tile(image_one_hot_labels, [1, fmnist_shape[1], fmnist_shape[2], 1])

    # noise for the generator to predict on
    noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, input_dim))

    # Now we can get the images from the generator
    # Steps: 1) Combine the noise vectors and the one-hot labels for the generator
    #        2) Generate the conditioned fake images
       
    noise_and_labels = combine_vectors(noise, one_hot_labels)
    fakeBatch = conditional_generator.predict(noise_and_labels)
    print(fakeBatch.shape)
    print(imageBatch.shape)

    
    imageBatch = np.moveaxis(imageBatch, 1, 3)
    # print(fakeBatch.shape)
    fake_and_label_channels = combine_vectors(fakeBatch, image_one_hot_labels, dim=-1)
    # fake_and_label_channels = np.moveaxis(fake_and_label_channels, 1, 3)
    real_and_label_channels = combine_vectors(imageBatch, image_one_hot_labels, dim=-1)
    # real_and_label_channels = np.moveaxis(real_and_label_channels, 1, 3)
    #print(noise_and_labels.shape)
    print(fake_and_label_channels.shape)
    print(real_and_label_channels.shape)
    print(type(real_and_label_channels))

    mix_images_batch = np.concatenate((real_and_label_channels,fake_and_label_channels),-1)

    fake_labels = tf.zeros_like(fake_and_label_channels)
    real_labels = tf.ones_like(real_and_label_channels)
    
    mix_labels_batch = real_labels + fake_labels
    mix_labels_batch = np.reshape(mix_labels_batch, (-1,))
    (mix_images_batch, mix_labels_batch) = shuffle(mix_images_batch, mix_labels_batch)
    discLoss = conditional_discriminator.train_on_batch(mix_images_batch, mix_labels_batch)
    #print(fake.shape)
    #print(noise_and_labels.shape)

    # Now you can get the predictions from the discriminator
    # Steps: 1) Create the input for the discriminator
    #           a) Combine the fake images with image_one_hot_labels, 
    #              remember to detach the generator (.detach()) so you do not backpropagate through it
    #           b) Combine the real images with image_one_hot_labels
    #        2) Get the discriminator's prediction on the fakes as disc_fake_pred
    #        3) Get the discriminator's prediction on the reals as disc_real_pred
    gen_noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, input_dim))
    gen_noise_and_labels = combine_vectors(gen_noise, one_hot_labels)
    # print(imageBatch.shape)
    gan_output = conditional_generator.predict(gen_noise_and_labels)
    update_gan(gan_output,labelsBatch)

    # check to see if this is the end of an epoch, and if so,
    # initialize the output path
    if i == batchesPerEpoch - 1:
      # print("[INFO] Step {}_{}: discriminator_loss={:.6f}, "
      # "adversarial_loss={:.6f}".format(epoch + 1, i,
      # discLoss, ganLoss))
      # size of the latent space
      latent_dim = generator_input_dim

      # generate samples
      n_samples = 25
      X, _ = generate_fake_samples(conditional_generator, latent_dim, n_samples)
      # plot the generated samples
      for i in range(n_samples):
        # define subplot
        pyplot.subplot(5, 5, 1 + i)
        # turn off axis labels
        pyplot.axis('off')
        # plot single image
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
      # show the figure
      pyplot.show()

[INFO] starting epoch 1 of 50...
(128, 28, 28, 1)
(128, 1, 28, 28)
(128, 28, 28, 11)
(128, 28, 28, 11)
<class 'tensorflow.python.framework.ops.EagerTensor'>


ValueError: ignored

In [ ]:
repeat([[1, 2], [3, 4]], repeats=[2, 3], axis=1)

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 1, 2, 2, 2],
       [3, 3, 4, 4, 4]], dtype=int32)>

In [ ]:
import torch
import tensorflow
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchsummary import summary

t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]
print(concat([t1, t2], 1))
t1 = torch.Tensor(t1)
t2 = torch.Tensor(t2)
print(torch.cat((t1,t2),1))


tf.Tensor(
[[ 1  2  3  7  8  9]
 [ 4  5  6 10 11 12]], shape=(2, 6), dtype=int32)
tensor([[ 1.,  2.,  3.,  7.,  8.,  9.],
        [ 4.,  5.,  6., 10., 11., 12.]])


In [ ]:
x = torch.Tensor([[1,2],[3,4]])
y = tensorflow.constant([[1,2],[3,4]])

x = x.repeat(2,2)
print(x)
print(x.shape)

y = tile(y,[2,2])
print(y)

tensor([[1., 2., 1., 2.],
        [3., 4., 3., 4.],
        [1., 2., 1., 2.],
        [3., 4., 3., 4.]])
torch.Size([4, 4])
tf.Tensor(
[[1 2 1 2]
 [3 4 3 4]
 [1 2 1 2]
 [3 4 3 4]], shape=(4, 4), dtype=int32)


In [ ]:
labelsBatch = labels[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
# imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
# initialize an (empty) output path
one_hot_labels = get_one_hot_labels(labelsBatch, n_classes)
image_one_hot_labels = one_hot_labels[:, None, None, :]
image_one_hot_labels = tile(image_one_hot_labels, [1, fmnist_shape[1], fmnist_shape[2], 1])
print(image_one_hot_labels.shape)

(128, 28, 28, 10)


In [ ]:
one = tf.ones_like(real_and_label_channels)
zero = tf.zeros_like(fake_and_label_channels)
mix = np.concatenate((one,zero),-1)
print(one.shape)
print(zero.shape)
print(mix.shape)

(128, 28, 28, 11)
(128, 28, 28, 11)
(128, 28, 28, 22)
